In [14]:
import pandas as pd


data_ls = []
name=['EN','AL','CN','SG']

for i in name:
    file = "./"+i+"/train"
    file_object = open(file, "r",encoding='UTF-8')
    dataset=[]
    for line in file_object:
        dataset.append(line)
    data=[]
    for i in range(len(dataset)):
        temp = dataset[i].split()
        if (len(temp)==2):
            data.append(temp)
    data_ls.append(pd.DataFrame(data, columns = ['Word', 'Tag']))

EN = data_ls[0]
AL = data_ls[1]
CN = data_ls[2]
SG = data_ls[3]

In [15]:
for i in range(len(data_ls)):
    print(name[i])
    print(data_ls[i].head())
    print("\n")

EN
        Word     Tag
0  Municipal    B-NP
1      bonds    I-NP
2        are    B-VP
3  generally  B-ADVP
4          a  B-ADJP


AL
  Word         Tag
0    小  B-DISTRICT
1    港  I-DISTRICT
2    四       B-POI
3    方       I-POI
4    家       I-POI


CN
  Word Tag
0    这   O
1   大概   O
2    是   O
3   所有   O
4   版本   O


SG
    Word         Tag
0    I'm           O
1  about           O
2     to           O
3    see           O
4   Adam  B-positive




In [16]:
for i in range(len(data_ls)):
    print(name[i])
    print(data_ls[i]['Tag'].value_counts())
    print("\n")

EN
I-NP       54591
B-NP       47305
O          23872
B-PP       18387
B-VP       18261
I-VP       10159
B-ADVP      3565
B-SBAR      1899
B-ADJP      1751
I-ADJP       574
B-PRT        468
I-ADVP       363
I-PP         223
I-CONJP       64
B-CONJP       49
I-SBAR        48
B-INTJ        26
B-LST         11
I-INTJ         7
I-UCP          4
B-UCP          1
Name: Tag, dtype: int64


AL
I-POI          28472
I-ROAD         15178
I-DISTRICT     13364
I-TOWN         10748
I-CITY         10430
I-PROV          8666
B-POI           7348
B-DISTRICT      6856
B-ROAD          6324
B-CITY          5697
B-ROADNO        5061
I-ROADNO        4666
B-TOWN          4612
I-SUBPOI        4557
B-PROV          4488
B-REDUNDANT     4137
I-REDUNDANT     3668
B-HOUSENO       3455
I-COMMUNITY     3148
B-ROOMNO        3107
I-HOUSENO       3045
I-PERSON        2232
I-DEVZONE       1834
B-SUBPOI        1650
I-CELLNO        1488
B-COMMUNITY     1479
B-CELLNO        1326
I-FLOORNO       1237
B-FLOORNO       1229
I-

In [17]:
def emission(df,word,tag):
    is_tag = (df.Tag==tag)
    y = is_tag.sum()
    xy = (df.Word[is_tag]==word).sum()
    return xy/y

In [18]:
print(emission(EN,"are","B-VP"))

0.03707354471277586


In [19]:
def smoothen(df,k):
    ls = df['Word'].value_counts()[df['Word'].value_counts()<k].index.tolist()
    df.loc[df['Word'].isin(ls), 'Word'] = "#UNK#"
    return df

EN=smoothen(EN,k=3)
AL=smoothen(AL,k=3)
CN=smoothen(CN,k=3)
SG=smoothen(SG,k=3)

In [20]:
for i in range(len(data_ls)):
    print(name[i])
    print(data_ls[i].head(20))
    print("\n")

EN
         Word     Tag
0       #UNK#    B-NP
1       bonds    I-NP
2         are    B-VP
3   generally  B-ADVP
4           a  B-ADJP
5         bit  I-ADJP
6       #UNK#  I-ADJP
7        than    B-PP
8   corporate    B-NP
9       bonds    I-NP
10         in    B-PP
11          a    B-NP
12  recession    I-NP
13          ,       O
14        but       O
15        not  B-ADJP
16         as  I-ADJP
17       safe  I-ADJP
18         as    B-PP
19      bonds    B-NP


AL
    Word         Tag
0      小  B-DISTRICT
1      港  I-DISTRICT
2      四       B-POI
3      方       I-POI
4      家       I-POI
5      园       I-POI
6     17   B-HOUSENO
7      幢   I-HOUSENO
8   1363    B-CELLNO
9      室    I-CELLNO
10     嘉      B-CITY
11     兴      I-CITY
12     南  B-DISTRICT
13     湖  I-DISTRICT
14     区  I-DISTRICT
15     文      B-ROAD
16     昌      I-ROAD
17     路      I-ROAD
18     新       B-POI
19     文       I-POI


CN
     Word        Tag
0       这          O
1      大概          O
2       是          O


In [21]:
from tqdm import tqdm_notebook
tqdm_notebook().pandas()


EN_count_series = EN.groupby(['Word', 'Tag']).size()
AL_count_series = AL.groupby(['Word', 'Tag']).size()
CN_count_series = CN.groupby(['Word', 'Tag']).size()
SG_count_series = SG.groupby(['Word', 'Tag']).size()

EN_tag_count = EN['Tag'].value_counts()
AL_tag_count = AL['Tag'].value_counts()
CN_tag_count = CN['Tag'].value_counts()
SG_tag_count = SG['Tag'].value_counts()

EN_count_u = pd.Series(EN_count_series.index.get_level_values('Tag').tolist(), index=EN_count_series.index).progress_apply(lambda x: EN_tag_count[x])
AL_count_u = pd.Series(AL_count_series.index.get_level_values('Tag').tolist(), index=AL_count_series.index).progress_apply(lambda x: AL_tag_count[x])
CN_count_u = pd.Series(CN_count_series.index.get_level_values('Tag').tolist(), index=CN_count_series.index).progress_apply(lambda x: CN_tag_count[x])
SG_count_u = pd.Series(SG_count_series.index.get_level_values('Tag').tolist(), index=SG_count_series.index).progress_apply(lambda x: SG_tag_count[x])

EN_count_series = EN_count_series / EN_count_u
AL_count_series = AL_count_series / AL_count_u
CN_count_series = CN_count_series / CN_count_u
SG_count_series = SG_count_series / SG_count_u


count_series=[EN_count_series, AL_count_series, CN_count_series, SG_count_series]

In [22]:
for i in range(len(count_series)):
    print(name[i])
    print(count_series[i])
    print("\n")

EN
Word     Tag   
!        O         0.000628
#        B-ADJP    0.001713
         B-NP      0.000423
         I-NP      0.000183
         O         0.000042
                     ...   
young    I-NP      0.000128
younger  B-NP      0.000085
         I-NP      0.000092
your     B-NP      0.000803
         I-NP      0.000037
Length: 12171, dtype: float64


AL
Word   Tag        
#UNK#  B-ASSIST       0.004994
       B-CELLNO       0.003771
       B-CITY         0.003160
       B-COMMUNITY    0.031778
       B-DEVZONE      0.004988
                        ...   
龙      I-SUBROAD      0.002801
       I-TOWN         0.002512
龚      B-POI          0.000680
       I-POI          0.000035
       I-ROAD         0.000066
Length: 13781, dtype: float64


CN
Word  Tag       
!     I-neutral     0.000551
      I-positive    0.000884
      O             0.000554
"     B-negative    0.001883
      B-positive    0.000273
                      ...   
［     O             0.000021
］     O             0.0

In [23]:
# def argmax(df,word):
#     if (word==""):
#         return ""
#     tags = (df.Tag[df.Word==word]).value_counts().index.tolist()
#     if (len(tags)==0):
#         tags = (df.Tag[df.Word=="#UNK#"]).value_counts().index.tolist()
#     emission_ls=[]
#     for i in tags:
#         emission_ls.append(emission(df,word,i))
#     return(tags[emission_ls.index(max(emission_ls))])

def argmax(df,word):
    if (word==""):
        return ""
    try:
        value = df.loc[word].idxmax()
    except:
        value = df.loc["#UNK#"].idxmax()
    
    return (value)

# def argmax(df,word):
#     if (word==""):
#         return ""
#     tags = pd.DataFrame((df.Tag[df.Word==word]).value_counts().index.tolist(), columns=['Tags'])
#     if (len(tags.index)==0):
#         tags = pd.DataFrame((df.Tag[df.Word=="#UNK#"]).value_counts().index.tolist(), columns=['Tags'])
#         word="#UNK#"
#     tags['Emission'] = tags['Tags'].apply(lambda x: emission(df,word,x))
#     return(tags['Tags'].loc[tags['Emission'].idxmax()])

In [24]:
test_ls = []
count=0
for i in name:
    file = "./"+i+"/dev.in"
    file_object = open(file, "r",encoding='UTF-8')
    ls=[]
    for line in file_object:
        ls.append(line.strip())
    df = pd.DataFrame(ls, columns = ['Word'])
    df['Tag'] = df['Word'].progress_apply(lambda x: argmax(count_series[count], x))
    test_ls.append(df)
    count+=1

EN_test = test_ls[0]
AL_test = test_ls[1]
CN_test = test_ls[2]
SG_test = test_ls[3]

In [25]:
for i in range(len(test_ls)):
    print(name[i])
    print(test_ls[i])
    print("\n")

EN
           Word     Tag
0           HBO    B-NP
1           has    B-VP
2         close  B-ADJP
3            to    B-PP
4            24    I-NP
...         ...     ...
27220      were    B-VP
27221        in    B-PP
27222  Congress    B-NP
27223         .       O
27224                  

[27225 rows x 2 columns]


AL
      Word          Tag
0        杭       B-CITY
1        州       I-CITY
2        市       I-CITY
3        西     B-ASSIST
4        湖   I-DISTRICT
...    ...          ...
26150    油     I-SUBPOI
26151    站     I-SUBPOI
26152    电  B-OTHERINFO
26153    联  I-OTHERINFO
26154                  

[26155 rows x 2 columns]


CN
      Word         Tag
0        一  I-negative
1       觉醒  B-negative
2       来天  B-negative
3        都           O
4        黑   B-neutral
...    ...         ...
24220   甜蜜  B-positive
24221    的           O
24222   浪漫           O
24223    。           O
24224                 

[24225 rows x 2 columns]


SG
                          Word         Tag
0        

In [26]:
test_out_ls=[]
for i in name:
    file = "./"+i+"/dev.out"
    file_object = open(file, "r",encoding='UTF-8')
    ls=[]
    for line in file_object:
        temp=line.split()
        if (len(temp)==0):
            temp.append("")
            temp.append("")
        ls.append(temp)
    test_out_ls.append(pd.DataFrame(ls, columns = ['Word', 'Tag']))

EN_test_out = test_out_ls[0]
AL_test_out = test_out_ls[1]
CN_test_out = test_out_ls[2]
SG_test_out = test_out_ls[3]

In [27]:
for i in range(len(test_out_ls)):
    print(name[i])
    print(test_out_ls[i])
    print("\n")

EN
           Word   Tag
0           HBO  B-NP
1           has  B-VP
2         close  B-NP
3            to  I-NP
4            24  I-NP
...         ...   ...
27220      were  B-VP
27221        in  B-PP
27222  Congress  B-NP
27223         .     O
27224                

[27225 rows x 2 columns]


AL
      Word          Tag
0        杭       B-CITY
1        州       I-CITY
2        市       I-CITY
3        西   B-DISTRICT
4        湖   I-DISTRICT
...    ...          ...
26150    油        I-POI
26151    站        I-POI
26152    电  B-REDUNDANT
26153    联  I-REDUNDANT
26154                  

[26155 rows x 2 columns]


CN
      Word Tag
0        一   O
1       觉醒   O
2       来天   O
3        都   O
4        黑   O
...    ...  ..
24220   甜蜜   O
24221    的   O
24222   浪漫   O
24223    。   O
24224         

[24225 rows x 2 columns]


SG
                          Word        Tag
0                         Tour  B-neutral
1                     Scotland  I-neutral
2                    followers          O
3   

In [28]:
for i in range(len(name)):
    file = "./"+name[i]+"/dev.prediction"
    test_ls[i].to_csv(file, sep=" ", index=False, header=False)

In [29]:
!python3 ./EvalScript/evalResult.py ./EN/dev.out ./EN/dev.prediction

'python3' is not recognized as an internal or external command,
operable program or batch file.


In [30]:
!python3 ./EvalScript/evalResult.py ./AL/dev.out ./AL/dev.prediction

'python3' is not recognized as an internal or external command,
operable program or batch file.


In [31]:
!python3 ./EvalScript/evalResult.py ./CN/dev.out ./CN/dev.prediction

'python3' is not recognized as an internal or external command,
operable program or batch file.


In [32]:
!python3 ./EvalScript/evalResult.py ./SG/dev.out ./SG/dev.prediction

'python3' is not recognized as an internal or external command,
operable program or batch file.
